Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [3]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-11-06 23:24:40--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.139.213
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.139.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  45.4MB/s    in 4.7s    

2019-11-06 23:24:45 (41.7 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [4]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [5]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01



## Join Data Practice

**These are the top 10 most frequently order products. How many time was each ordered?**

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

In [4]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


In [0]:
# Import Everything
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.options.display.max_rows=999
pd.options.display.max_columns=100

In [0]:
# Keeping my notebook DRY
def checkData(filename):
  return pd.read_csv(f'{filename}.csv').sample(5)

In [9]:
checkData('aisles')

,aisle_id,aisle
64,65,protein meal replacements
124,125,trail mix snack mix
20,21,packaged cheese
84,85,food storage
88,89,salad dressing toppings


In [15]:
checkData('orders')

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
2746364,2609484,165479,prior,44,3,9,2.0
678433,1844563,40898,prior,11,0,19,30.0
1026874,3229582,61738,prior,46,2,16,8.0
280670,1810151,16971,prior,17,1,17,8.0
1275025,1957387,76678,prior,57,4,11,3.0


In [11]:
checkData('departments')

,department_id,department
11,12,meat seafood
17,18,babies
10,11,personal care
4,5,alcohol
2,3,bakery


In [14]:
checkData('order_products__train')

,order_id,product_id,add_to_cart_order,reordered
370745,905030,8587,1,0
301643,734750,13517,15,1
457294,1119519,25075,7,0
650948,1601863,11710,1,1
1264270,3123549,3988,10,1


In [12]:
checkData('order_products__prior')

,order_id,product_id,add_to_cart_order,reordered
13249593,1398281,23720,12,0
920061,97243,34022,6,0
19478174,2054423,48028,1,1
22624487,2386294,260,1,0
23369397,2464758,14814,8,1


In [16]:
checkData('products')

,product_id,product_name,aisle_id,department_id
17479,17480,Pure-Castile Soap 18-in-1 Hemp Tea Tree,25,11
42524,42525,IncrediBites for Small Dogs,40,8
37187,37188,Organic Lori's Lemon Tea,94,7
30447,30448,Just Honeycrisp Apples,92,18
42177,42178,"White Tea, White Peony",94,7


### 1. Write down which columns you need and which dataframes have them.

> product_name - products.csv

> product_id - order_products__prior

> order_id - order_products_prior

In [0]:
# Make a df I can call for the products csv
prod_df = pd.read_csv('products.csv')

In [0]:
# Same for order_products_prior csv
opp_df = pd.read_csv('order_products__prior.csv')

### 2. Merge these into a single dataframe.

In [10]:
# Merged the to dataframes together inner so it only keep the like values I don't think the added NaN will be helpful
fullord = pd.merge(opp_df, prod_df, on='product_id', how='inner')
fullord.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16


In [0]:
zoomFullOrd = fullord.drop(['add_to_cart_order', 'reordered', 'aisle_id', 'department_id'], axis=1)

### 3. Use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [22]:
# I need the 10 most frequently ordered products represented as n
n = 10
# On the product_name column give me all the unique(in order by most called) values up to the n(10)th one then display it in a list
zoomFullOrd['product_name'].value_counts()[:n].index.tolist()

['Banana',
 'Bag of Organic Bananas',
 'Organic Strawberries',
 'Organic Baby Spinach',
 'Organic Hass Avocado',
 'Organic Avocado',
 'Large Lemon',
 'Strawberries',
 'Limes',
 'Organic Whole Milk']

## Reshape Data Section
- Complete the code cells we skipped near the beginning of the notebook
- Table 2 --> Tidy
- Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

**1. Replicate the lesson code.**

In [30]:
# Replicated the lesson code.

myindex=['John Smith', 'Jane Doe', 'Mary Johnson']
mycolumns=['treatmenta', 'treatmentb']

table1 = pd.DataFrame(
            [[np.nan, 2],
             [16, 11],
             [3, 1]],
          columns=mycolumns,
        index=myindex
)
table1

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


In [31]:
table1 = table1.reset_index()
table1

,index,treatmenta,treatmentb
0,John Smith,NaN,2
1,Jane Doe,16.0,11
2,Mary Johnson,3.0,1


In [33]:
tidy = table1.melt(id_vars='index', value_vars=['treatmenta', 'treatmentb'])
tidy

,index,variable,value
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [34]:
tidy.rename(columns={'index':'Name', 'variable':'Trt', 'value':'Results'})

,Name,Trt,Results
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


**2. Tidy --> Table 2**

### Flights dataset pivot table

**1. Load flights dataset**

In [0]:
flights = sns.load_dataset('flights')

**2. Create pivots table showing the number of passengers by months and year. index = year | column = months**

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####